In [1]:
import pandas as pd
import numpy as np
# import os
from dotenv import load_dotenv
from datetime import datetime

In [2]:
# #csv sports_io data pull if the api is not utilized

# data_dict = {}
# csv_path = f'../data/sports_io/'

# for i in os.listdir(csv_path):
#     data_df = pd.read_csv(csv_path + i)
    
#     data_dict[i.split('.')[0]] = data_df

# print(data_dict.keys())

In [3]:
#sports_io nba documentation link:  https://fantasydata.com/api/api-documentation/nba#
#data_set_list = ['GamesByDate', 'GameOddsByDate', 'GameOddsLineMovement', 'Schedules', 'Stadiums', 'TeamGameStatsByDate', 'TeamSeasonStats']
#odds_data_set_dict = {'GamesByDate' : 'YYYY-MM-DD', 'GameOddsByDate' : 'YYYY-MM-DD', 'GameOddsLineMovement' : 'GameID', 'Schedules' : 'Season', 'Stadiums' : NaN, 'TeamGameStatsByDate' : 'YYYY-MM-DD', 'TeamSeasonStats' : 'Season'}

#assign sports_io api key
load_dotenv()
sports_io_api_key = os.getenv('sports_io_api_key')


#assign sports_io url to variable that can be pulled with different parameters
def sports_io_api_pull(data_set, data_set_parameter, sports_io_api_key):
    sports_io_odds_url = f"https://api.sportsdata.io/api/nba/odds/json/{data_set}/{data_set_parameter}?key={sports_io_api_key}"

    response_data = pd.read_json(sports_io_odds_url)

    return response_data

In [4]:
# date_list = pd.date_range(start = '1/2/2020', periods = 1).strftime(('%Y-%m-%d'))

# odds_data_set = 'GamesByDate'
# odds_list = []

# for i in date_list:

#     odds_data = sports_io_api_pull(odds_data_set, i, sports_io_api_key)
#     odds_list.append(odds_data)

# odds_df = pd.concat(odds_list, axis = 0)
# odds_df = odds_df.loc[(odds_df['SeasonType'] != 2), :]
# odds_df = schedules_df.loc[(schedules_df['Status'] == 'Final') | (schedules_df['Status'] == 'F/OT')]

# odds_df = odds_df[['GameID', 'Day', 'HomeTeam', 'HomeTeamID', 'AwayTeam', 'AwayTeamID', 'HomeTeamScore', 'AwayTeamScore', 'PointSpread', 'OverUnder']]
# odds_df['Day'] = pd.to_datetime(odds_df['Day'])

In [5]:
odds_data_set = 'Games'
season_list = [*range(2020, 2021, 1)]

odds_list = []

for i in season_list:
    odds_data = sports_io_api_pull(odds_data_set, i, sports_io_api_key)
    odds_list.append(odds_data)

odds_df = pd.concat(odds_list, axis = 0)
odds_df = odds_df.loc[(odds_df['SeasonType'] != 2), :]
odds_df = odds_df.loc[(odds_df['Status'] == 'Final') | (odds_df['Status'] == 'F/OT')]

odds_df = odds_df[['GameID', 'Day', 'HomeTeam', 'HomeTeamID', 'AwayTeam', 'AwayTeamID', 'HomeTeamScore', 'AwayTeamScore', 'PointSpread', 'OverUnder']]
odds_df['Day'] = pd.to_datetime(odds_df['Day'])

In [6]:
move_data_set = 'GameOddsLineMovement'
game_list = list(odds_df['GameID'])

move_list = []

for i in game_list:

    move_data = sports_io_api_pull(move_data_set, i, sports_io_api_key)

    move_data['PregameOdds'] = move_data['PregameOdds'].apply(pd.DataFrame)
    move_data['Opening_PointSpread'] = move_data['PregameOdds'].apply(lambda x: x.loc[x['Created'] == x['Created'].min(), 'HomePointSpread'])
    move_data['Opening_OverUnder'] = move_data['PregameOdds'].apply(lambda x: x.loc[x['Created'] == x['Created'].min(), 'OverUnder'])
    move_data = move_data[['GameId', 'Opening_PointSpread', 'Opening_OverUnder']]

    move_list.append(move_data)

move_df = pd.concat(move_list).reset_index(drop = True)
move_df = move_df.rename(columns = {'GameId' : 'GameID'})

In [11]:
lines_df = odds_df.merge(move_df, how = 'inner', on = 'GameID')

lines_df = lines_df.dropna()

lines_df.head()

,GameID,Day,HomeTeam,HomeTeamID,AwayTeam,AwayTeamID,HomeTeamScore,AwayTeamScore,PointSpread,OverUnder,Opening_PointSpread,Opening_OverUnder
25,13929,2019-10-26,MIL,15,MIA,4,126.0,131.0,-11.5,223.0,-9.5,226.5
27,13931,2019-10-26,ATL,3,ORL,5,103.0,99.0,2.5,216.5,0.0,219.0
28,13932,2019-10-26,NY,6,BOS,9,95.0,118.0,5.5,218.5,-1.0,211.0
29,13933,2019-10-26,CHI,11,TOR,10,84.0,108.0,3.0,219.0,3.5,223.0
30,13934,2019-10-26,CLE,12,IND,13,110.0,99.0,4.5,213.5,5.5,214.0


In [12]:
old_col = lines_df.columns

home_col = ['game_id', 'day', 'team', 'team_id', 'opp', 'opp_id', 'team_score', 'opp_score', 'spread', 'o/u', 'opening_spread', 'opening_o/u']
home_col_dict = dict(zip(old_col, home_col))

home_df = lines_df.rename(columns = home_col_dict)
home_df['home'] = 1

away_col = ['game_id', 'day', 'opp', 'opp_id', 'team', 'team_id', 'opp_score', 'team_score', 'spread', 'o/u', 'opening_spread', 'opening_o/u']
away_col_dict = dict(zip(old_col, away_col))

away_df = lines_df.rename(columns = away_col_dict)
away_df['home'] = 0
away_df['spread'] = away_df['spread'] * -1
away_df['opening_spread'] = away_df['opening_spread'] * -1

In [15]:
games_df = pd.concat([home_df, away_df])

games_df['favored_spread'] = np.where(games_df['spread'] <= 0, 1, 0)

games_df['score_spread'] = games_df['team_score'] - games_df['opp_score']
games_df['cover_spread'] = np.where(-games_df['score_spread'] < games_df['spread'], 1, 0)

games_df['score_o/u'] = games_df['team_score'] + games_df['opp_score']
games_df['cover_o/u'] = np.where(games_df['score_o/u'] > games_df['o/u'], 1, 0)

games_df['move_spread'] = games_df['spread'] - games_df['opening_spread']
games_df['move_spread_direction'] = np.where(games_df['move_spread'] <= 0, 1, 0)

games_df['move_o/u'] = games_df['o/u'] - games_df['opening_o/u']
games_df['move_o/u_direction'] = np.where(games_df['move_o/u'] >= 0, 1, 0)

games_df = games_df[['game_id', 'day', 'team', 'team_id', 'opp', 'opp_id', 'home', 'favored_spread', 'opening_spread', 'spread', 'move_spread', 'move_spread_direction', 'cover_spread', 'opening_o/u', 'o/u', 'move_o/u', 'move_o/u_direction', 'cover_o/u']]

games_df = games_df.sort_values(by = ['team', 'day'])

mask_team = games_df.duplicated(['team'])
games_df['days_between_games'] = np.where(mask_team, pd.to_numeric(games_df['day'].diff().dt.days, downcast = 'integer'), 5)

games_df['travel_between_games'] = np.where(mask_team, games_df['home'] != games_df['home'].shift(), np.where(games_df['home'] == 1, 0, 1))

games_df = games_df.reset_index(drop = True)

games_df.head()

,game_id,day,team,team_id,opp,opp_id,home,favored_spread,opening_spread,spread,move_spread,move_spread_direction,cover_spread,opening_o/u,o/u,move_o/u,move_o/u_direction,cover_o/u,days_between_games,travel_between_games
0,13931,2019-10-26,ATL,3,ORL,5,1,0,0.0,2.5,2.5,0,1,219.0,216.5,-2.5,0,0,5.0,0
1,13955,2019-10-29,ATL,3,MIA,4,0,0,7.5,8.5,1.0,0,0,224.5,223.5,-1.0,0,0,3.0,1
2,13969,2019-10-31,ATL,3,MIA,4,1,0,6.5,6.5,0.0,1,0,216.5,215.0,-1.5,0,0,2.0,1
3,14022,2019-11-08,ATL,3,SAC,30,1,0,-1.0,2.5,3.5,0,0,219.5,225.5,6.0,1,1,8.0,0
4,14041,2019-11-10,ATL,3,POR,19,0,0,8.5,9.0,0.5,0,0,228.0,226.5,-1.5,0,1,2.0,1


In [16]:
games_df.to_csv(f'../../data/sports_io/api_data_pull/nba_odds_data_clean.csv', index = False)